In [ ]:
import pulp as plp
from ALB_instance_tools import *
import numpy as np
import pandas as pd
import os

In [ ]:
! pwd


In [ ]:
'20_249.alb'.split(".")[0].split("_")[1]

In [ ]:

#Gets list of all instance (.alb) files in the SALBP_benchmark/small\ data\ set_n\=20 folder
instance_list = get_instance_list('SALBP_benchmark/small data set_n=20')

#sorts instance list by instance number
instance_list = sorted(instance_list, key=lambda k: int(k['name'].split("_")[1]))


In [ ]:
parse_alb(instance_list[517]['location'])

In [ ]:

def define_ALBP_1_problem(instance, max_stations = 20):
    prob = plp.LpProblem("ALPB_1", plp.LpMinimize)
    #creating decision variables
    tasks = plp.LpVariable.dicts("task_o_s", (instance['task_times'].keys(), range(1,max_stations + 1)), cat='Binary')
    #objective function
    prob += plp.lpSum([ station * tasks[task][station] for station in range(1,max_stations + 1) for task in instance['task_times'].keys()])
    #definining constraints
    #constraint 1 only choose 1 station for each task
    for task in instance['task_times'].keys():
        prob += plp.lpSum([tasks[task][station] for station in range(1,max_stations + 1)]) == 1
    #constraint 2 task and station assignment must respect takt time
    for station in range(1,max_stations + 1):
        prob += plp.lpSum([instance['task_times'][task] * tasks[task][station] for task in instance['task_times'].keys()]) <= instance['cycle_time']
    #constraint 3 tasks must respect precedence constraints
    for precedence in instance['precedence_relations']:
        prob += plp.lpSum([station * tasks[precedence[0]][station] for station in range(1,max_stations + 1)]) <= plp.lpSum([station * tasks[precedence[1]][station] for station in range(1,max_stations + 1)])
    return prob

def solve_ALBP_1(instance):
    prob = define_ALBP_1_problem(instance)
    prob.solve(solver=plp.XPRESS_PY( msg=False))
 
    return prob




In [ ]:
def get_ALBP_solutions(problems_list):
    solutions = []
    for problem in problems_list:
        instance = parse_alb(problem['location'])
        prob = solve_ALBP_1(instance)
        #creates a new dictionary entry that contains the data on the instances
        print('solving problem', problem['name'])
        entry = {'name':problem['name']}
        entry['no_tasks'] = len(instance['task_times'].keys())
        entry['order_strength'] = instance['order_strength']
        entry['cycle_time'] = instance['cycle_time']
        max_station = -10
        for variable in prob.variables():
            
            if variable.varValue > 0:
                station = int(variable.name.split("_")[4])

                if station > max_station:
                    max_station = station
        entry['no_stations'] = max_station
        solutions.append(entry)
    return solutions
#solution_outputs = get_ALBP_solutions(instance_list)

In [ ]:
# solution_outputs = pd.DataFrame(solution_outputs)
# solution_outputs.to_csv('ALBP_1_solutions.csv')

In [ ]:
#Scoring functions
def task_time_weight(task, instance):
    return instance['task_times'][task]


     


#fills up each station with available tasks in order of score
def insert_task(tasks_dict, station_capacities, assignment_dict):
    for index, station in enumerate(station_capacities):
            for task in tasks_dict.keys():
                if instance['task_times'][task] <= station and all(predecessor not in tasks_dict.keys() for predecessor in tasks_dict[task]['predecessors']):
                    station_capacities[index] -= instance['task_times'][task]
                    assignment_dict[task] = index + 1
                    tasks_dict.pop(task)
                    return

# RA heuristic as described in "A comparative Evaluation of Heuristics for the Assembly Line Balancing Problem" by Ponnanbalam et. al              
def rank_and_assign(score_function, instance, max_stations = 20):
    station_capacities = [instance['cycle_time'] for i in range(0, max_stations)]
    tasks_dict = {}
    task_assignment = {}
    for task in instance['task_times'].keys():
        task_dict = {}
        task_dict['score'] = score_function(task, instance)
        task_dict['predecessors'] = [precedence[0] for precedence in instance['precedence_relations'] if precedence[1] == task]

        tasks_dict[task] = task_dict
    #sorts tasks_dict by score
    tasks_dict = {k: v for k, v in sorted(tasks_dict.items(), key=lambda item: item[1]['score'])}
    #Inserts tasks into stations until there are no more tasks
    while len(tasks_dict.keys()) > 0:
        insert_task(tasks_dict, station_capacities, task_assignment)
    return station_capacities, task_assignment

instance = parse_alb(instance_list[0]['location'])

station_capacities1, task_assignment1 = rank_and_assign(task_time_weight,instance)
station_capacities1

In [ ]:
#inserts tasks into first available station
def insert_task_iuff(tasks_dict, station_capacities, assignment_dict, available_tasks):
    for task in tasks_dict.keys():
        if task in available_tasks:
            for index, station in enumerate(station_capacities):
                if instance['task_times'][task] <= station:
                    station_capacities[index] -= instance['task_times'][task]
                    assignment_dict[task] = index + 1
                    del tasks_dict[task]
                    return

def update_tasks(tasks_dict, available_tasks, task_assignment):
    for task in tasks_dict.keys():
        if task not in available_tasks:
            if all(predecessor not in task_assignment.keys() for predecessor in tasks_dict[task]['predecessors']):
                available_tasks.append(task)
 #TODO I need to make sure we are adding tasks in the right order, based on the score
    
     
# IUFF heuristic as described in "A comparative Evaluation of Heuristics for the Assembly Line Balancing Problem" by Ponnanbalam et. al              
def immediate_update_first_fit(score_function, instance, max_stations = 20):
    station_capacities = [instance['cycle_time'] for i in range(0, max_stations)]
    tasks_dict = {}
    task_assignment = {}
    available_tasks = []
    for task in instance['task_times'].keys():
        task_dict = {}
        task_dict['score'] = score_function(task, instance)
        task_dict['predecessors'] = [precedence[0] for precedence in instance['precedence_relations'] if precedence[1] == task]
        #adds tasks with no predecessors to available tasks
        if not task_dict['predecessors']:
            available_tasks.append(task)
        tasks_dict[task] = task_dict
    #sorts tasks_dict by score
    tasks_dict = {k: v for k, v in sorted(tasks_dict.items(), key=lambda item: item[1]['score'], reverse=True)}
    #Inserts first available tasks into a station

    while len(task_assignment) < instance['num_tasks']:
        print(task_assignment)
        insert_task_iuff(tasks_dict, station_capacities, task_assignment, available_tasks)
        update_tasks(tasks_dict, available_tasks, task_assignment)
    return station_capacities, task_assignment
instance = parse_alb(instance_list[0]['location'])
station_capacities, task_assignment = immediate_update_first_fit(task_time_weight,instance)

In [ ]:
station_capacities

In [ ]:
task_assignment1

In [ ]:
instance

In [ ]:
task_assignment